In [ ]:
# Import standard Python modules.
import datetime
import importlib
import os
import platform
import sys

# Import 3rd-party modules.
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

# Import TensorFlow.
import tensorflow as tf

In [ ]:
# Use 64-bit math in TensorFlow.
tf.keras.backend.set_floatx("float64")

# Case 4: Compressible, isothermal, $\frac {dP} {dx}$ variable

Consider one-dimensional, isothermal, compressible fluid flow in a channel. For simplicity, assume the channel is a pipe with a circular cross-section of diameter $D$. Flow in the channel is controlled by the Bernoulli equation:

\begin{equation}
    P + \frac {1} {2} \rho u^2 + \rho g h = constant = C_1
\end{equation}

where $P$ is the pressure, $\rho$ is the mass density, $u$ is the fluid flow speed, $g$ is the acceleration due to gravity, and $h$ is the height of the fluid.

Neglecting gravity, this equation can be rearranged to solve for the flow speed $u$ as a function of the pressure $P$:

\begin{equation}
    u = \left[ \frac {2} {\rho} \left( C_1 - P \right) \right]^{\frac {1} {2}}
\end{equation}

The pressure and density are related by the ideal gas law:

\begin{equation}
    P = n k T = \frac {\rho} {m} k T
\end{equation}

where $n$ is the number density, $m$ is the mass of a single fluid particle, $k$ is the Boltzmann constant, and $T$ is the temperature.

Rewriting the speed equation with this relation gives:

\begin{equation}
    u = \left[ \frac {2 k T} {m P} \left( C_1- P \right) \right]^{\frac {1} {2}} \\
    = C_3 \left( \frac {C_1} {P} - 1 \right)^{\frac {1} {2}}
\end{equation}

where the constant $C_3$ is given by:

\begin{equation}
    C_3 = \left( \frac {2 k T} {m} \right)^{\frac {1} {2}}
\end{equation}

The change in speed is computed using the chain rule:

\begin{equation}
    \frac {du} {dx} = \frac {du} {dP} \frac {dP} {dx} \\
    = C_3 \frac {1} {2} \left( \frac {C_1} {P} - 1 \right)^{- \frac {1} {2}} \left( - \frac {C_1} {P^2} \right) \frac {dP} {dx} \\
    = - \frac {C_1 C_3} {2} \left[ P^3 \left( C_1 - P \right) \right] ^{- \frac {1} {2}} \frac {dP} {dx}
\end{equation}

where $x$ is the linear distance along the channel.

Now assume the the pressure gradient $dP/dx$ is a function of the speed $u(x)$. The functional form uses the Darcy friction factor $f_D$:

\begin{equation}
    \frac {dP} {dx} = - \frac {f_D \rho} {2 D} u^2
\end{equation}

where $D$ is the pipe diameter.

We now have a coupled set of ordinary differential equations for $u(x)$ and $P(x)$.

Now assume $\rho_0 = 1$, $P_0 = 1$, $u_0 = 1$, and $\frac {f_D} {D} = 1$, and select $m$ and $T$ such that $C_3 = 1$. The constant $C_1 = \frac {3} {2}$, and the differential equations become:

\begin{equation}
    \frac {du} {dx} = - \frac {3} {4} \left[ P^3 \left( \frac {3} {2} - P \right) \right] ^{-1/2} \frac {dP} {dx} \\
    \frac {dP} {dx} = - \frac {1} {2} u^2
\end{equation}

This system of ODEs can be solved numerically.

In [ ]:
eq_name = "case4"

# Define the differential equations to be numerically integrated.
def dY_dt(Y, x):
    (u, P) = Y
    dP_dx = -0.5*P*u**2
    du_dx = -3/2*(P**3*(3 - 2*P))**(-0.5)*dP_dx
    return [du_dx, dP_dx]

# Compute the numerical solutions.
nx = 10001
xn = np.linspace(0, 1, nx)
Y0 = [1, 1]  # Initial conditions
Y = odeint(dY_dt, Y0, xn)
un = Y[:, 0]
Pn = Y[:, 1]

# Compute the derivatives of the numerical solutions.
dun = np.gradient(un)
dPn = np.gradient(Pn)
dxn = np.gradient(xn)
dun_dx = dun/dxn
dPn_dx = dPn/dxn

# Plot the numerical solutions and derivatives.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xn, un, label="$u$")
ax.plot(xn, Pn, label="$P$")
ax.plot(xn, dun_dx, label="$du/dx$")
ax.plot(xn, dPn_dx, label="$dP/dx$")
ax.set_xlabel("x")
ax.set_ylabel("$u$, $P$, $du/dx$, or $dP/dx$")
ax.grid()
ax.legend()
ax.set_title("Numerical solutions and derivatives for %s" % eq_name)
plt.show()

In [ ]:
def print_system_information():
    print("System report:")
    print(datetime.datetime.now())
    print("Host name: %s" % platform.node())
    print("OS: %s" % platform.platform())
    print("uname:", platform.uname())
    print("Python version: %s" % sys.version)
    print("Python build:", platform.python_build())
    print("Python compiler: %s" % platform.python_compiler())
    print("Python implementation: %s" % platform.python_implementation())
    # print("Python file: %s" % __file__)

In [ ]:
def create_output_directory(path=None):
    path_noext, ext = os.path.splitext(path)
    output_dir = path_noext
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [ ]:
from nnde.math.trainingdata import create_training_grid2

def create_training_data(*n_train):
    x_train = np.linspace(0, 0.9, n_train[0])
    return x_train

In [ ]:
def build_model(H, w0_range, u0_range, v0_range):
    hidden_layer_1 = tf.keras.layers.Dense(
        units=H, use_bias=True,
        activation=tf.keras.activations.sigmoid,
        kernel_initializer=tf.keras.initializers.RandomUniform(*w0_range),
        bias_initializer=tf.keras.initializers.RandomUniform(*u0_range)
    )
    output_layer = tf.keras.layers.Dense(
        units=1,
        activation=tf.keras.activations.linear,
        kernel_initializer=tf.keras.initializers.RandomUniform(*v0_range),
        use_bias=False,
    )
    model = tf.keras.Sequential([hidden_layer_1, output_layer])
    return model

In [ ]:
print_system_information()

In [ ]:
# Set up the output directory.
eq_name = "case4"
path = os.path.join(".", eq_name)
output_dir = create_output_directory(path)

In [ ]:
# Define the hyperparameters.

# Training optimizer
optimizer_name = "Adam"

# Initial parameter ranges
w0_range = [-0.1, 0.1]
u0_range = [-0.1, 0.1]
v0_range = [-0.1, 0.1]

# Maximum number of training epochs.
max_epochs = 40000

# Learning rate.
learning_rate = 0.01

# Absolute tolerance for consecutive loss function values to indicate convergence.
tol = 1e-6

# Number of hidden nodes.
H = 10

# Number of dimensions
m = 1

# Number of training points in each dimension.
nx_train = 11
n_train = nx_train

# Random number generator seed.
random_seed = 0

In [ ]:
# Create and save the training data.
x_train = create_training_data(nx_train)
np.savetxt(os.path.join(output_dir,'x_train.dat'), x_train)

In [ ]:
# Define the differential equations using TensorFlow operations.

@tf.function
def ode_u(x, u, P, du_dx, dP_dx):
    G = du_dx + 3/2*(P**3*(3 - 2*P))**(-0.5)*dP_dx
    return G

@tf.function
def ode_P(x, u, P, du_dx, dP_dx):
    G = dP_dx + 0.5*P*u**2
    return G

In [ ]:
# Define the trial functions.

@tf.function
def Y_trial_u(x, N):
    A = tf.constant([[1.0]], dtype="float64")
    P = x
    Y = A + P*N
    return Y

@tf.function
def Y_trial_P(x, N):
    A = tf.constant([[1.0]], dtype="float64")
    P = x
    Y = A + P*N
    return Y

In [ ]:
# Build the models.
model_u = build_model(H, w0_range, u0_range, v0_range)
model_P = build_model(H, w0_range, u0_range, v0_range)

# Create the optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Create history variables.
losses_u = []
losses_P = []
losses = []
phist_u = []
phist_P = []

# Set the random number seed for reproducibility.
tf.random.set_seed(random_seed)

# Rename the training data Variable for convenience, just for training.
# shape (n_train, m)
x_train_var = tf.Variable(np.array(x_train).reshape((n_train, m)), name="x_train")
x = x_train_var

# Clear the convergence flag to start.
converged = False

# Train the model.

print("Hyperparameters: n_train = %s, H = %s, max_epochs = %s, optimizer = %s, learning_rate = %s"
      % (n_train, H, max_epochs, optimizer_name, learning_rate))
t_start = datetime.datetime.now()
print("Training started at", t_start)

for epoch in range(max_epochs):

    # Run the forward pass.
    with tf.GradientTape(persistent=True) as tape1:
        with tf.GradientTape(persistent=True) as tape0:

            # Compute the network outputs at the training points.
            N_u = model_u(x)
            N_P = model_P(x)

            # Compute the trial solutions.
            u = Y_trial_u(x, N_u)
            P = Y_trial_P(x, N_P)

        # Compute the gradients of the trial solutions wrt inputs.
        du_dx = tape0.gradient(u, x)
        dP_dx = tape0.gradient(P, x)

        # Compute the estimates of the differential equations.
        G_u = ode_u(x, u, P, du_dx, dP_dx)
        G_P = ode_P(x, u, P, du_dx, dP_dx)

        # Compute the loss functions.
        L_u = tf.math.sqrt(tf.reduce_sum(G_u**2)/n_train)
        L_P = tf.math.sqrt(tf.reduce_sum(G_P**2)/n_train)
        L = L_u + L_P

    # Save the current losses.
    losses_u.append(L_u)
    losses_P.append(L_P)
    losses.append(L.numpy())

    # Check for convergence.
    if epoch > 0:
        loss_delta = losses[-1] - losses[-2]
        if abs(loss_delta) <= tol:
            converged = True
            break

    # Compute the gradient of the loss function wrt the network parameters.
    pgrad_u = tape1.gradient(L, model_u.trainable_variables)
    pgrad_P = tape1.gradient(L, model_P.trainable_variables)

    # Save the parameters used in this epoch.
    phist_u.append(
        np.hstack(
            (model_u.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
             model_u.trainable_variables[1].numpy(),       # u (H,) row vector
             model_u.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
        )
    )
    phist_P.append(
        np.hstack(
            (model_P.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
             model_P.trainable_variables[1].numpy(),       # u (H,) row vector
             model_P.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
        )
    )

    # Update the parameters for this epoch.
    optimizer.apply_gradients(zip(pgrad_u, model_u.trainable_variables))
    optimizer.apply_gradients(zip(pgrad_P, model_P.trainable_variables))

    if epoch % 100 == 0:
        print("Ending epoch %s, loss function = %f" % (epoch, L.numpy()))

# Save the parameters used in the last epoch.
phist_u.append(
    np.hstack(
        (model_u.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
         model_u.trainable_variables[1].numpy(),       # u (H,) row vector
         model_u.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
    )
)
phist_P.append(
    np.hstack(
        (model_P.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
         model_P.trainable_variables[1].numpy(),       # u (H,) row vector
         model_P.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
    )
)

n_epochs = epoch + 1

t_stop = datetime.datetime.now()
print("Training stopped at", t_stop)
t_elapsed = t_stop - t_start
print("Total training time was %s seconds." % t_elapsed.total_seconds())
print("Epochs: %d" % n_epochs)
print("Final value of loss function: %f" % losses[-1])
print("converged = %s" % converged)

# Save the parameter and loss function histories.
np.savetxt(os.path.join(output_dir, 'phist_u.dat'), np.array(phist_u))
np.savetxt(os.path.join(output_dir, 'phist_P.dat'), np.array(phist_P))
np.savetxt(os.path.join(output_dir, 'losses.dat'), np.array(losses))
np.savetxt(os.path.join(output_dir, 'losses_u.dat'), np.array(losses_u))
np.savetxt(os.path.join(output_dir, 'losses_P.dat'), np.array(losses_P))

In [ ]:
# Compute and save the trained results at training points.
with tf.GradientTape(persistent=True) as tape:
    N_u = model_u(x)
    N_P = model_P(x)
    ut_train = Y_trial_u(x, N_u)
    Pt_train = Y_trial_P(x, N_P)
dut_dx_train = tape.gradient(ut_train, x)
dPt_dx_train = tape.gradient(Pt_train, x)
np.savetxt(os.path.join(output_dir, 'ut_train.dat'), ut_train.numpy().reshape((n_train,)))
np.savetxt(os.path.join(output_dir, 'Pt_train.dat'), Pt_train.numpy().reshape((n_train,)))
np.savetxt(os.path.join(output_dir, 'dut_dx_train.dat'), dut_dx_train.numpy())
np.savetxt(os.path.join(output_dir, 'dPt_dx_train.dat'), dPt_dx_train.numpy())

# Compute and save the numerical solutions and derivatives at training points.
un_train = np.interp(x_train, xn, un)
Pn_train = np.interp(x_train, xn, Pn)
dun_dx_train = np.interp(x_train, xn, dun_dx)
dPn_dx_train = np.interp(x_train, xn, dPn_dx)
np.savetxt(os.path.join(output_dir, 'un_train.dat'), un_train)
np.savetxt(os.path.join(output_dir, 'Pn_train.dat'), Pn_train)
np.savetxt(os.path.join(output_dir, 'dun_dx_train.dat'), dun_dx_train)
np.savetxt(os.path.join(output_dir, 'dPn_dx_train.dat'), dPn_dx_train)

# Compute and save the error in the trained solutions and derivatives at training points.
ut_err_train = ut_train.numpy().reshape((nx_train,)) - un_train
Pt_err_train = Pt_train.numpy().reshape((nx_train,)) - Pn_train
dut_dx_err_train = dut_dx_train.numpy().reshape((nx_train,)) - dun_dx_train
dPt_dx_err_train = dPt_dx_train.numpy().reshape((nx_train,)) - dPn_dx_train
np.savetxt(os.path.join(output_dir, 'ut_err_train.dat'), ut_err_train)
np.savetxt(os.path.join(output_dir, 'Pt_err_train.dat'), Pt_err_train)
np.savetxt(os.path.join(output_dir, 'dut_dx_err_train.dat'), dut_dx_err_train)
np.savetxt(os.path.join(output_dir, 'dPt_dx_err_train.dat'), dPt_dx_err_train)

# Compute the final RMS errors in the solutions at the training points.
ut_rmse_train = np.sqrt(np.sum(ut_err_train**2)/n_train)
Pt_rmse_train = np.sqrt(np.sum(Pt_err_train**2)/n_train)
print("ut_rmse_train = %s" % ut_rmse_train)
print("Pt_rmse_train = %s" % Pt_rmse_train)

In [ ]:
# Plot the loss function histories.
plt.semilogy(losses, label="L (total)")
plt.semilogy(losses_u, label="L (u)")
plt.semilogy(losses_P, label="L (P)")
plt.xlabel("Epoch")
plt.ylabel("Loss function")
plt.legend()
plt.grid()
plt.title("Loss function evolution for %s\n%s optimizer, $\eta$=%s, H=%s, $n_{train}$=%s, epochs=%s" %
          (eq_name, optimizer_name, learning_rate, H, n_train, n_epochs))
plt.show()

In [ ]:
# Plot the the trained solutions and derivatives at the training points.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_train, ut_train, label="$u_t$")
ax.plot(x_train, Pt_train, label="$P_t$")
ax.plot(x_train, dut_dx_train, label="$du_t/dx$")
ax.plot(x_train, dPt_dx_train, label="$dP_t/dx$")
ax.set_xlabel("x")
ax.set_ylabel("$u_t$ or $du_t/dx$")
ax.grid()
ax.legend()
ax.set_title("Trained solutions and derivatives for %s" % eq_name)
plt.show()

In [ ]:
# Plot the errors in the trained solutions and derivatives at the training points.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_train, ut_err_train, label="$u_t - u_n$")
ax.plot(x_train, Pt_err_train, label="$P_t - P_n$")
ax.plot(x_train, dut_dx_err_train, label="$du_t/dx - du_a/dx$")
ax.plot(x_train, dPt_dx_err_train, label="$dP_t/dx - dP_a/dx$")
ax.set_xlabel("x")
ax.set_ylabel("Trained - numerical$")
ax.grid()
ax.legend()
ax.set_title("Error in trained solution for %s" % eq_name)
plt.show()